# Eatery density difference between two datasets

In [1]:
import contextily as ctx
import geopandas as gpd
#import matplotlib as mpl
import matplotlib.animation as animation
import matplotlib.cm as colormaps
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from IPython.display import HTML
from descartes import PolygonPatch
from matplotlib import colors
from matplotlib.collections import PatchCollection
from shapely import wkt
#https://geopandas.readthedocs.io/en/latest/gallery/plotting_basemap_background.html

%matplotlib inline

#sudo apt-get install libgeos++-dev libproj-dev ffmpeg yasm
#sudo pip3 install geopandas descartes Cython Proj contextily

In [2]:
df = pd.read_csv('OSM-NYC-differential-eatery-density.csv',sep=",")
gdf = gpd.GeoDataFrame(df, geometry=df['geometry'].apply(wkt.loads))
gdf.head()

,oc,fc,ratio,logratio,geohash,geometry
0,7,0,0.125000,-0.903090,dr5rztbn6qbg4hm7kbqn,"POLYGON ((-73.845 40.765, -73.845 40.775, -73...."
1,315,0,0.003165,-2.499687,dr5rt5p2bn53jnpr24g6,"POLYGON ((-73.94500000000011 40.7049999999999,..."
2,4,0,0.200000,-0.698970,dr5rwhu7hnjtd2sftjpe,"POLYGON ((-73.9050000000001 40.715, -73.905000..."
3,461,0,0.002165,-2.664642,dr5rk9cmtj3g967n8r2p,"POLYGON ((-73.97500000000009 40.6549999999999,..."
4,11,0,0.083333,-1.079181,dr5xcmfq6qv5hs2er0qw,"POLYGON ((-73.7650000000002 40.765, -73.765000..."


In [3]:
gdf.crs = {'init': 'epsg:4326'}
gdf_proj = gdf.to_crs(epsg=3857)
gdf_proj.head()

,oc,fc,ratio,logratio,geohash,geometry
0,7,0,0.125000,-0.903090,dr5rztbn6qbg4hm7kbqn,POLYGON ((-8220387.797629287 4977740.767001763...
1,315,0,0.003165,-2.499687,dr5rt5p2bn53jnpr24g6,POLYGON ((-8231519.746708626 4968926.125064375...
2,4,0,0.200000,-0.698970,dr5rwhu7hnjtd2sftjpe,POLYGON ((-8227066.967076894 4970394.680194238...
3,461,0,0.002165,-2.664642,dr5rk9cmtj3g967n8r2p,POLYGON ((-8234859.331432423 4961586.654469891...
4,11,0,0.083333,-1.079181,dr5xcmfq6qv5hs2er0qw,POLYGON ((-8211482.238365847 4977740.767001763...


## Build iterations

In [4]:
metricN = 0.01
BANDWIDTH=50

results = []
oc_gen = gdf_proj.oc
fc_gen = gdf_proj.fc
oc_gen_norm = np.divide(oc_gen, np.sum(oc_gen))

# prepare for iteration
fc_genThis = fc_gen

for i in range(2,2000+1):
    fc_genNext = np.copy(fc_genThis)

    fc_genThis_delta = np.subtract(oc_gen,fc_genThis)
    fc_genThis_deltaWeight = np.divide(fc_genThis_delta, np.sum(fc_genThis_delta))
    candidateW = np.where( (fc_genThis_delta > 0), fc_genThis_deltaWeight, 0 )
    if(len(candidateW) == 0):
        print("Terminal condition 1")
        break
    candidateW = candidateW/np.sum(candidateW)
    
    for k in np.random.choice(range(0,len(fc_genThis_delta)),BANDWIDTH,True,candidateW):
        fc_genNext[k] = fc_genNext[k] + 1

    fc_genThis_norm = np.divide(fc_genThis, np.sum(fc_genThis))    
    metric = np.mean(fc_genThis_delta)
    results.append(fc_genThis_delta)

    print (i,
           np.sum(fc_genThis),np.sum(oc_gen),
           np.sum(fc_genThis<oc_gen),
           np.sum(fc_genThis>oc_gen),
           metric
          )
    
    fc_genThis = np.copy(fc_genNext)
    
    if metric < metricN:
        print ("Terminal condition 2")
        break

2 165 69256 529 0 130.60680529300566
3 215 69256 529 0 130.51228733459357
4 265 69256 529 0 130.41776937618147
5 315 69256 529 0 130.32325141776937
6 365 69256 529 0 130.22873345935727
7 415 69256 529 0 130.13421550094517
8 465 69256 529 0 130.03969754253308
9 515 69256 529 0 129.94517958412098
10 565 69256 529 0 129.85066162570888
11 615 69256 529 0 129.75614366729678
12 665 69256 529 0 129.66162570888469
13 715 69256 529 0 129.5671077504726
14 765 69256 529 0 129.4725897920605
15 815 69256 529 0 129.3780718336484
16 865 69256 529 0 129.2835538752363
17 915 69256 529 0 129.1890359168242
18 965 69256 529 0 129.0945179584121
19 1015 69256 529 0 129.0
20 1065 69256 529 0 128.9054820415879
21 1115 69256 529 0 128.8109640831758
22 1165 69256 529 0 128.7164461247637
23 1215 69256 529 0 128.6219281663516
24 1265 69256 529 0 128.5274102079395
25 1315 69256 529 0 128.4328922495274
26 1365 69256 529 0 128.33837429111531
27 1415 69256 529 0 128.24385633270322
28 1465 69256 529 0 128.149338374291

281 14115 69256 529 0 104.23629489603024
282 14165 69256 529 0 104.14177693761815
283 14215 69256 529 0 104.04725897920605
284 14265 69256 529 0 103.95274102079395
285 14315 69256 528 0 103.85822306238185
286 14365 69256 528 0 103.76370510396976
287 14415 69256 528 0 103.66918714555766
288 14465 69256 528 0 103.57466918714556
289 14515 69256 528 0 103.48015122873346
290 14565 69256 528 0 103.38563327032136
291 14615 69256 528 0 103.29111531190927
292 14665 69256 528 0 103.19659735349717
293 14715 69256 528 0 103.10207939508507
294 14765 69256 528 0 103.00756143667297
295 14815 69256 528 0 102.91304347826087
296 14865 69256 528 0 102.81852551984878
297 14915 69256 528 0 102.72400756143668
298 14965 69256 528 0 102.62948960302458
299 15015 69256 528 0 102.53497164461248
300 15065 69256 527 0 102.44045368620039
301 15115 69256 527 0 102.34593572778827
302 15165 69256 527 0 102.25141776937618
303 15215 69256 527 0 102.15689981096408
304 15265 69256 527 0 102.06238185255198
305 15315 69256 

513 25715 69256 526 0 82.30812854442344
514 25765 69256 526 0 82.21361058601134
515 25815 69256 526 0 82.11909262759924
516 25865 69256 526 0 82.02457466918715
517 25915 69256 526 0 81.93005671077505
518 25965 69256 526 0 81.83553875236295
519 26015 69256 526 0 81.74102079395085
520 26065 69256 526 0 81.64650283553875
521 26115 69256 526 0 81.55198487712666
522 26165 69256 526 0 81.45746691871456
523 26215 69256 526 0 81.36294896030246
524 26265 69256 526 0 81.26843100189036
525 26315 69256 526 0 81.17391304347827
526 26365 69256 526 0 81.07939508506617
527 26415 69256 526 0 80.98487712665407
528 26465 69256 526 0 80.89035916824197
529 26515 69256 526 0 80.79584120982987
530 26565 69256 526 0 80.70132325141778
531 26615 69256 526 0 80.60680529300568
532 26665 69256 526 0 80.51228733459357
533 26715 69256 526 0 80.41776937618147
534 26765 69256 526 0 80.32325141776937
535 26815 69256 526 0 80.22873345935727
536 26865 69256 526 0 80.13421550094517
537 26915 69256 526 0 80.03969754253308


766 38365 69256 519 0 58.39508506616257
767 38415 69256 519 0 58.300567107750474
768 38465 69256 519 0 58.20604914933838
769 38515 69256 519 0 58.11153119092628
770 38565 69256 519 0 58.01701323251418
771 38615 69256 519 0 57.92249527410208
772 38665 69256 519 0 57.82797731568998
773 38715 69256 519 0 57.73345935727788
774 38765 69256 519 0 57.63894139886578
775 38815 69256 519 0 57.544423440453684
776 38865 69256 519 0 57.449905482041586
777 38915 69256 519 0 57.35538752362949
778 38965 69256 519 0 57.26086956521739
779 39015 69256 519 0 57.16635160680529
780 39065 69256 519 0 57.071833648393195
781 39115 69256 519 0 56.9773156899811
782 39165 69256 519 0 56.882797731569
783 39215 69256 519 0 56.7882797731569
784 39265 69256 519 0 56.6937618147448
785 39315 69256 519 0 56.599243856332706
786 39365 69256 518 0 56.50472589792061
787 39415 69256 518 0 56.41020793950851
788 39465 69256 518 0 56.315689981096405
789 39515 69256 518 0 56.22117202268431
790 39565 69256 517 0 56.12665406427221

1013 50715 69256 491 0 35.04914933837429
1014 50765 69256 491 0 34.954631379962194
1015 50815 69256 490 0 34.860113421550096
1016 50865 69256 490 0 34.765595463138
1017 50915 69256 490 0 34.6710775047259
1018 50965 69256 490 0 34.5765595463138
1019 51015 69256 490 0 34.482041587901705
1020 51065 69256 490 0 34.3875236294896
1021 51115 69256 490 0 34.2930056710775
1022 51165 69256 490 0 34.198487712665404
1023 51215 69256 490 0 34.103969754253306
1024 51265 69256 490 0 34.00945179584121
1025 51315 69256 490 0 33.91493383742911
1026 51365 69256 490 0 33.82041587901701
1027 51415 69256 490 0 33.725897920604915
1028 51465 69256 490 0 33.63137996219282
1029 51515 69256 490 0 33.53686200378072
1030 51565 69256 490 0 33.44234404536862
1031 51615 69256 490 0 33.34782608695652
1032 51665 69256 490 0 33.253308128544425
1033 51715 69256 490 0 33.15879017013233
1034 51765 69256 490 0 33.06427221172023
1035 51815 69256 489 0 32.96975425330813
1036 51865 69256 489 0 32.87523629489603
1037 51915 6925

1255 62815 69256 418 0 12.175803402646503
1256 62865 69256 418 0 12.081285444234405
1257 62915 69256 418 0 11.986767485822305
1258 62965 69256 417 0 11.892249527410208
1259 63015 69256 417 0 11.79773156899811
1260 63065 69256 415 0 11.703213610586012
1261 63115 69256 414 0 11.608695652173912
1262 63165 69256 414 0 11.514177693761814
1263 63215 69256 413 0 11.419659735349716
1264 63265 69256 413 0 11.325141776937619
1265 63315 69256 413 0 11.23062381852552
1266 63365 69256 413 0 11.136105860113421
1267 63415 69256 413 0 11.041587901701323
1268 63465 69256 413 0 10.947069943289225
1269 63515 69256 413 0 10.852551984877127
1270 63565 69256 413 0 10.758034026465028
1271 63615 69256 411 0 10.66351606805293
1272 63665 69256 411 0 10.568998109640832
1273 63715 69256 411 0 10.474480151228734
1274 63765 69256 410 0 10.379962192816635
1275 63815 69256 408 0 10.285444234404537
1276 63865 69256 407 0 10.190926275992439
1277 63915 69256 407 0 10.096408317580341
1278 63965 69256 405 0 10.00189035916

## Plot animation

In [5]:
# check data range
np.min(np.array(results)),np.max(np.array(results))

(-2, 1970)

In [6]:
fig, ax = plt.subplots(figsize=(9,9))
plt.axis('off');

plt.suptitle('Animate eatery density difference between two datasets', fontsize=18)

xmin, ymin, xmax, ymax = gdf_proj.geometry.total_bounds
basemap, extent = ctx.bounds2img(xmin, ymin, xmax, ymax, zoom=10,
                                 url='http://tile.stamen.com/terrain/tileZ/tileX/tileY.png')
ax.imshow(basemap, extent=extent, interpolation='bilinear')
ax.axis((xmin, xmax, ymin, ymax))

im = None
def updatefig(i):
    global im, ax, results
    if im:
        im.remove()
    ax.set_title('Iteration %d' % (i+1), fontsize=18)

    rr = np.log10(np.where(results[i]>0,results[i],1))
    bounds=[
        np.mean(rr)-1.0*np.std(rr),
        np.mean(rr),
        np.mean(rr)+1.0*np.std(rr), 
        np.mean(rr)+2.0*np.std(rr)           
    ]
    cc = ['blue','red']
    
    cmap = colormaps.coolwarm
    norm = colors.Normalize(vmin=-0.3, vmax=4)
    
    patches = [PolygonPatch(geom) for geom in gdf_proj.geometry]
    coll = PatchCollection(patches, cmap=cmap, norm=norm, alpha=0.5, edgecolor='None')
    coll.set_array(results[i])
    im = ax.add_collection(coll)
    
    return

# 30s animation (30'000 ms)
anim = animation.FuncAnimation(fig, updatefig, frames=len(results), interval=30000/len(results))
plt.close(anim._fig)
HTML(anim.to_html5_video())

In [7]:
np.min(results[-1]),np.max(results[-1])

(-2, 2)